In [1]:
import pandas as pd
import numpy as np
from collections import *
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import treebank
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from pymagnitude import *
import time
from scipy.spatial.distance import cosine
import warnings
warnings.filterwarnings('ignore')

In [2]:
nltk.download('averaged_perceptron_tagger')
nltk.download('treebank')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\arjun\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\arjun\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [ ]:
data = pd.read_csv('data/questions.csv')
data = data
path = 'data/GoogleNews-vectors-negative300.magnitude'
vectors = Magnitude(path)
tok = RegexpTokenizer('\w+')
data.question1 = pd.Series([tok.tokenize(str(s)) for s in data.question1])
data.question2 = pd.Series([tok.tokenize(str(s)) for s in data.question2])

In [ ]:
# helper functions and constants
def sum_vec(words):
    res = np.sum([vectors.query(w) for w in words if w in vectors], axis=0)
    if type(res) is np.float64:
        return np.array([1e-32] * vectors.dim)
    else:
        return res
    
def rm_stop(words):
    stop_words = nltk.corpus.stopwords.words('english')
    return [w for w in words if not w in stop_words]

def train_test(col1 ,col2):
    sep = int(len(data)*.75)
    ret = {}
    ret['train_pnts'] = data.as_matrix([col1, col2])[:sep]
    ret['train_pnts'] = [[cosine(x[0], x[1])] for x in ret['train_pnts']]
    ret['train_tgts'] = data.as_matrix(["is_duplicate"])[:sep]
    ret['test_pnts']  = data.as_matrix([col1, col2])[sep:]
    ret['test_pnts']  = [[cosine(x[0], x[1])] for x in ret['test_pnts']]
    ret['test_tgts']  = data.as_matrix(["is_duplicate"])[sep:]
    return ret

i_tup_num = 0
i_question1 = 3
i_question2 = 4
i_is_duplicate = 5
i_vec1 = 6
i_vec2 = 7
i_stop1 = 8
i_stop_vec1 = 9
i_stop2 = 10
i_stop_vec2 = 11

In [ ]:
t = time.time()
vec1 = pd.Series(sum_vec(words) for words in data.question1)
data.insert(len(data.columns), "vec1", vec1)
print("Finished vec1")
vec2 = pd.Series(sum_vec(words) for words in data.question2)
data.insert(len(data.columns), "vec2", vec2)
print("Finished vec2")
print(time.time() - t)
t = time.time()
stop1 = pd.Series(rm_stop(words) for words in data.question1)
data.insert(len(data.columns), "stop1", stop1)
stop_vec1 = pd.Series(sum_vec(words) for words in data.stop1)
data.insert(len(data.columns), "stop_vec1", stop_vec1)
print("Finished stop1")
stop2 = pd.Series(rm_stop(words) for words in data.question2)
data.insert(len(data.columns), "stop2", stop2)
stop_vec2 = pd.Series(sum_vec(words) for words in data.stop2)
data.insert(len(data.columns), "stop_vec2", stop_vec1)
print("Finished stop2")
print(time.time() - t)

In [ ]:
#initial data information
print("# of data points:", len(data))
print("# of word vectors:", len(vectors))
print('vector dimensions:', vectors.dim)
cnt = Counter(data.is_duplicate)
print("random baseline:", 1/len(cnt.keys()))
print("most common baseline:", max([cnt[k]/sum(cnt.values()) for k in cnt.keys()]))

In [ ]:
accuracy_score(
    [0 if nltk.translate.bleu_score.sentence_bleu(
    [data.question1[i]], 
    data.question2[i]) < .5 else 1
    for i in range(len(data))],
    data.is_duplicate)

In [ ]:
x = train_test("vec1", "vec2")
lr = LogisticRegression().fit(x['train_pnts'], x['train_tgts'])
print("normal embeddings LR accuracy:", accuracy_score(lr.predict(x['test_pnts']), x['test_tgts']))
print("normal embeddings LR coefficients:", lr.coef_)

In [ ]:
x = train_test("stop_vec1", "stop_vec2")
lr = LogisticRegression().fit(x['train_pnts'], x['train_tgts'])
print("no stop words LR accuracy:", accuracy_score(lr.predict(x['test_pnts']), x['test_tgts']))
print("no stop words LR coefficients:", lr.coef_)

In [ ]:
i = 0
for x in data.values:
    if x[i_question1] == x[i_question2]:
        print(x[i_is_duplicate], x[i_tup_num])
        print(x[i_question1])
        i += 1
print(i)

In [ ]:
hello = data.as_matrix(["vec1", "vec2"])
hello = [[cosine(x[0], x[1])] for x in hello]

In [ ]:
x = data.values
k = 0
for i in range(len(data.values)):
    if hello[i] == [0.0]:
        print(x[i][i_is_duplicate], x[i][i_tup_num])
        print(x[i][i_question1])
        print(x[i][i_question2])
        k += 1
print(k)

In [ ]:
nltk.corpus.stopwords.words('english')

In [ ]:
cosine(vectors.query("PhD"), vectors.query("Ph.D."))

In [ ]:
"WW1" in vectors

In [ ]:
"Chick-fil-A" in vectors

In [ ]:
data[299692:299692+1]

In [ ]:
from nltk.corpus import treebank

trees = treebank.parsed_sents()

In [ ]:
nltk.download('treebank')

In [ ]:
print(trees[0])

In [ ]:
x = 'dsafa'
x.hello = 1

In [ ]:
def filter(tree):
    child_nodes = [child.label() for child in tree
                   if isinstance(child, nltk.Tree)]
    return child_nodes

trees = [subtree for tree in treebank.parsed_sents() for subtree in tree.subtrees(filter)]

production_rules = defaultdict(list)
terminals = list()

for tree in trees:
    for prod in tree.productions():
        lhs = str(prod.lhs()).replace('=','-').replace('|','-')
        if prod.is_lexical():
            terminals.append(lhs)
        if prod.is_nonlexical():
            rhs = ''
            for s in prod.rhs(): 
                rhs += str(s).replace('=','-').replace('|','-') + ' '
            production_rules[lhs].append(str(rhs.strip()))
            
terminals = set(terminals)
terminals.remove("''")

production_counters = dict()

for prod in production_rules:
    production_counters[prod] = Counter(production_rules[prod])

def to_prod_rules(lhs, c):
    global terminals
    denom = sum(c.values())
    cfg_str = list()
    for rhs in c:
        rhs_prods = rhs.split()
        r = ''
        for rule in rhs_prods: 
            if rule == '' or rule == "''": continue
            r += "'" + rule + "' " if rule in terminals else rule + " "
        cfg_str.append("{} -> {} [{}]".format(lhs, r.strip(), '{:.20f}'.format(c[rhs] / denom)))
    return cfg_str

production_lhs = list(production_counters.keys())
production_lhs.remove('S')
production_lhs = ['S'] + production_lhs

prod_rules = list()

for lhs in production_lhs:
    prod_rules += to_prod_rules(lhs, production_counters[lhs])
    
prod_rules_str = '\n'.join(prod_rules)

grammar = nltk.PCFG.fromstring(prod_rules_str)

viterbi_parser = nltk.ViterbiParser(grammar)

In [ ]:
sentence = ['I', 'like', 'to', 'eat', 'tacos']

pos_sentence = nltk.pos_tag(sentence)

print(sentence)
print(pos_sentence)


for tree in viterbi_parser.parse([x[1] for x in pos_sentence]):
    print(tree)

In [ ]:
nltk.pos_tag(['I'])

In [ ]:
nltk.pos_tag(['i'])